<a href="https://colab.research.google.com/github/ynkim0/Stock_and_Financial_Statements/blob/main/Financial_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from io import BytesIO
import zipfile
import xmltodict

api = 'https://opendart.fss.or.kr/api/corpCode.xml'
res = requests.get(api, params={'crtfc_key': '9a360d3e3365eba4c46575c83fe03b7947088252'})
data_xml = zipfile.ZipFile(BytesIO(res.content))

https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf

In [ ]:
data_xml.namelist()

['CORPCODE.xml']

In [ ]:
import xmltodict
import json

data_xml = data_xml.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')

In [ ]:
for item in data:
    if item['corp_name'] in ["삼성전자", "SK하이닉스", "NAVER"]:
        print(item)

{'corp_code': '00126380', 'corp_name': '삼성전자', 'stock_code': '005930', 'modify_date': '20220509'}
{'corp_code': '00164779', 'corp_name': 'SK하이닉스', 'stock_code': '000660', 'modify_date': '20220331'}
{'corp_code': '00266961', 'corp_name': 'NAVER', 'stock_code': '035420', 'modify_date': '20220728'}


http://blog.quantylab.com/2020-10-09-dart.html

In [ ]:
crtfc_key = '9a360d3e3365eba4c46575c83fe03b7947088252'
 
import requests 
import pandas as pd 
import io 
import zipfile
import xml.etree.ElementTree as et 
import json

In [ ]:
def get_corpcode(crtfc_key): 
    """ 
    OpenDART 기업 고유번호 받아오기 
    return 값: 주식코드를 가진 업체의 DataFrame 
    """ 
    params = {'crtfc_key':crtfc_key} 
    items = ["corp_code","corp_name","stock_code","modify_date"] 
    item_names = ["고유번호","회사명","종목코드","수정일"] 
    url = "https://opendart.fss.or.kr/api/corpCode.xml" #요청 url
    res = requests.get(url,params=params) #url 불러오기
    zfile = zipfile.ZipFile(io.BytesIO(res.content))  #zip file 받기
    fin = zfile.open(zfile.namelist()[0])  #zip file 열고
    root = et.fromstring(fin.read().decode('utf-8'))  #utf-8 디코딩
    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: # 종목코드가 있는 경우 
            data.append([]) #data에 append하라 
            for item in items: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns=item_names) 
    return df

In [ ]:
stock_comp = get_corpcode(crtfc_key)
stock_comp

,고유번호,회사명,종목코드,수정일
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630
...,...,...,...,...
3515,00185505,제일바이오,052670,20221122
3516,00993931,동양파일,228340,20221124
3517,00115287,동방,004140,20221124
3518,00557933,진도,088790,20221108


In [ ]:
def Frame(url, items, item_names, params): 
    """
    url : json형태로 요청하는 주소
    items : 반환되는 데이터들의 key를 가진 리스트
    item_names :  데이터프레임을 만들때 컬럼명 리스트
    params : url 요청시 필수값으로 들어가는 인자들을 가진 딕셔너리
    """
    res = requests.get(url, params)
    json_data = res.json()
    json_dict = json.loads(res.text) 
    data = [] 
    if json_dict['status'] == "000":  # 오류 없이 정상적으로 데이터가 있다면 
        for line in json_dict['list']: 
            data.append([])
            for itm in items: 
                if itm in line.keys(): 
                    data[-1].append(line[itm]) 
                else: 
                    data[-1].append('')
    df = pd.DataFrame(data, columns=item_names)
    return df

In [ ]:
def get_단일회사재무제표(crtfc_key, corp_code, bsns_year, reprt_code, fs_div = "CFS"): 
    items = ["rcept_no","reprt_code","bsns_year","corp_code","sj_div","sj_nm", 
             "account_id","account_nm","account_detail","thstrm_nm", "thstrm_amount",
             "thstrm_add_amount","frmtrm_nm","frmtrm_amount", "frmtrm_q_nm","frmtrm_q_amount",
             "frmtrm_add_amount","bfefrmtrm_nm", "bfefrmtrm_amount","ord"] 
    item_names = ["접수번호","보고서코드","사업연도","고유번호","재무제표구분", "재무제표명",
                  "계정ID","계정명","계정상세","당기명","당기금액", "당기누적금액","전기명","전기금액","전기명(분/반기)", 
                  "전기금액(분/반기)","전기누적금액","전전기명","전전기금액", "계정과목정렬순서"] 
    params = {'crtfc_key':crtfc_key, 'corp_code':corp_code, 'bsns_year':bsns_year, 'reprt_code':reprt_code, 'fs_div':fs_div} 
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?" 
    df = Frame(url, items, item_names, params)
    return df

In [ ]:
#1개 해보기 
corp_code = stock_comp['고유번호']
final_reprt_df = get_fnlttSinglAcntAll(crtfc_key, corp_code[0], 2020, '11013', fs_div = "CFS") 
#반복
from tqdm import tqdm
for k in tqdm(range(1, len(stock_comp))):
    reprt_df = get_fnlttSinglAcntAll(crtfc_key, corp_code[k], 2020, '11013', fs_div = "CFS") 
    final_reprt_df = pd.concat([final_reprt_df, reprt_df], axis = 0)

NameError: ignored

In [ ]:
def sep(x, df): 
    df = df[df['재무제표구분'] == x]
    return df
 
final_BS = sep('BS', final_reprt_df) #재무상태표 
final_CIS = sep('CIS', final_reprt_df) #포괄손익계산서 
final_CF = sep('CF', final_reprt_df) #현금흐름표
final_SCE = sep('SCE', final_reprt_df) #자본변동표
final_IS = sep('IS', final_reprt_df) #손익계산서 